In [ ]:


import pandas as pd
import glob
import os

DAY_MAP = {'Mon': 0, 'Tue': 1, 'Wed': 2, 'Thu': 3, 'Fri': 4, 'Sat': 5, 'Sun': 6}

def process_file(filepath: str) -> pd.DataFrame:
    team = os.path.basename(filepath).split('_')[0].lower()
    
    df = pd.read_csv(filepath)
    
    df = df.rename(columns={
        'Unnamed: 2_level_0 Week':       'Week',
        'Unnamed: 3_level_0 Date':       'Date',
        'Unnamed: 4_level_0 Day':        'Day',
        'Unnamed: 5_level_0 Unnamed: 5_level_1': 'Home',
        'Unnamed: 6_level_0 Opp':        'Opp',
        'Score Pts':                     'Team Pts',
        'Score OT':                      'OT'
    })
    
    df = df[df['Week'].notna() & (df['Week'] != 0)]
    
    if 'Score Pts0' in df.columns:
        opp_col = 'Score Pts0'
    elif 'Score PtsO' in df.columns:
        opp_col = 'Score PtsO'
    else:
        raise KeyError("No opponent-points column found (Score Pts0/Score PtsO).")
    df['Opp Pts'] = df[opp_col]
    
    drop_cols = [
        'Unnamed: 0_level_0 Rk',
        'Unnamed: 1_level_0 Gtm',
        'Passing Yds.1',
        opp_col
    ]
    df = df.drop(columns=[c for c in drop_cols if c in df.columns])
    
    df['Team'] = team
    
    df['Year']    = df['Year'].astype(int)
    df['Week']    = df['Week'].astype(int)
    df['Date']    = pd.to_datetime(df['Date'])
    df['Day']     = df['Day'].map(DAY_MAP)
    df['Home']    = df['Home'].astype(str).str.contains('@').map({True: 0, False: 1})
    df['Score Rslt'] = df['Score Rslt'].astype(str)
    df['Team Pts']  = df['Team Pts'].astype(int)
    df['Opp Pts']   = df['Opp Pts'].astype(int)
    df['OT']        = df['OT'].astype(str).str.contains('OT').astype(int)
    
    dtype_map = {
        'Passing Cmp': int,    'Passing Att': int,  'Passing Cmp%': float,
        'Passing Yds': float,  'Passing TD': int,   'Passing Y/A': float,
        'Passing AY/A': float, 'Passing Rate': float,'Passing Sk': int,
        'Rushing Att': int,    'Rushing Yds': float,'Rushing TD': int,
        'Rushing Y/A': float,  'Total Offense Ply': int,
        'Total Offense Tot': float,'Total Offense Y/P': float,
        'Kicking FGA': int,    'Kicking FGM': int,
        'Kicking XPA': int,    'Kicking XPM': int,
        'Punting Pnt': int,    'Punting Yds': float,
        'First Downs Pass': int,'First Downs Rsh': int,
        'First Downs Pen': int,'First Downs 1stD': int,
        'Downs 3DConv': int,   'Downs 3DAtt': int,
        'Downs 4DConv': int,   'Downs 4DAtt': int,
        'Penalties Pen': int,  'Penalties Yds': float,
        'Turnovers FL': int,   'Turnovers Int': int,
        'Turnovers TO': int
    }
    for col, typ in dtype_map.items():
        if col in df.columns:
            df[col] = df[col].astype(typ)
    
    return df

def main():
    # <<< UPDATE THESE PATHS >>>
    input_folder  = '/Users/sethfried/Fantasy Football/data/team_game_logs'
    output_folder = '/Users/sethfried/Fantasy Football/Cleaned Data/Cleaned_Team_logs'
    
    os.makedirs(output_folder, exist_ok=True)
    all_dfs = []
    
    for csv_file in glob.glob(os.path.join(input_folder, '*.csv')):
        print(f"Processing {os.path.basename(csv_file)}")
        cleaned = process_file(csv_file)
        cleaned.to_csv(os.path.join(output_folder, os.path.basename(csv_file)), index=False)
        all_dfs.append(cleaned)
    
    if all_dfs:
        combined = pd.concat(all_dfs, ignore_index=True)
        combined.to_csv(os.path.join(output_folder, 'all_teams_clean.csv'), index=False)
        print("Combined CSV written.")

if __name__ == '__main__':
    main()


Processing den_2006_2024_gamelog.csv
Processing dal_2006_2024_gamelog.csv
Processing sea_2006_2024_gamelog.csv
Processing sdg_2006_2024_gamelog.csv
Processing tam_2006_2024_gamelog.csv
Processing cin_2006_2024_gamelog.csv
Processing nyj_2006_2024_gamelog.csv
Processing oti_2006_2024_gamelog.csv
Processing kan_2006_2024_gamelog.csv
Processing sfo_2006_2024_gamelog.csv
Processing atl_2006_2024_gamelog.csv
Processing nor_2006_2024_gamelog.csv
Processing pit_2006_2024_gamelog.csv
Processing gnb_2006_2024_gamelog.csv
Processing jax_2006_2024_gamelog.csv
Processing htx_2006_2024_gamelog.csv
Processing phi_2006_2024_gamelog.csv
Processing mia_2006_2024_gamelog.csv
Processing buf_2006_2024_gamelog.csv
Processing min_2006_2024_gamelog.csv
Processing rai_2006_2024_gamelog.csv
Processing was_2006_2024_gamelog.csv
Processing car_2006_2024_gamelog.csv
Processing det_2006_2024_gamelog.csv
Processing chi_2006_2024_gamelog.csv
Processing ram_2006_2024_gamelog.csv
Processing cle_2006_2024_gamelog.csv
P